## Agentic_Event_Planner

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Crew, Agent, Task

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [ ]:
# Crew Tools
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()  # Search Internet using Google
scrape_tool = ScrapeWebsiteTool() #Scarpes the infomation from the websites

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
# Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
        "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [ ]:
#Creating the Venue Pydantic Obj
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic), 
#Pydantic helps to take fuzzy input types to a stronger output types
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [12]:
#Tasks
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
    # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [13]:
logistics_task = Task(
    description="Coordinate catering and "
                "equipment for an event "
                "with {expected_participants} participants "
                "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [14]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [15]:
#Creating crew
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
        logistics_task, 
        marketing_task],
    
    verbose=True
)

2025-03-25 19:10:30,275 - 6284603392 - __init__.py-__init__:362 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(

In [17]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                        "and industry leaders "
                        "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
To find a suitable venue for the Tech Innovation Conference in San Francisco, I need to gather information on venues that can accommodate the event's theme, size, and budget constraints. I should start by searching for potential venues online.

Action: Search the internet
Action Input: {"search_query": "Tech conference venues San Francisco"} 


Search results: Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/tech-conferences/
Snippet: Tech conferences events in San Francisco, CA. Category. Business · Science & Tech · Music · Film & Media · Performing & Visual Arts · Fashion · Health ...
---
Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferen



> Entering new CrewAgentExecutor chain...

> Entering new CrewAgentExecutor chain...

Given the venue's capacity is 400 and our event has 500 participants, we need to address the capacity issue first. We might need to either find an alternative venue or make arrangements to accommodate the additional participants. I should consult the Venue Coordinator to explore potential solutions or alternative venues. 

Action: Ask question to co-worker
Action Input: {
  "coworker": "Venue Coordinator", 
  "question": "Our current venue, Shack 15, located at Ferry Building, San Francisco, has a capacity of 400, but we have 500 participants for our event on 2024-09-15. Could you suggest any solutions to accommodate the extra 100 participants, or should we consider an alternative venue?",
  "context": "We are planning an event with 500 participants, but the current venue's capacity is only 400. We need to either find a way to fit everyone comfortably or seek another venue."
}

> Entering new CrewAg

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
    data = json.load(f)

pprint(data)

{'address': 'Ferry Building, San Francisco',
 'booking_status': 'Available',
 'capacity': 400,
 'name': 'Shack 15'}


In [19]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md